# Configuration interaction

Multipsi provides a very general configuration code, allowing a wide variety of CI expansions, including FullCI and truncated CI (for example CI with single and double excitations CISD). Note however that the code is mostly designed around Full CI (for use in particular in complete active space CAS methods, see following chapters) and is thus not as efficient as a dedicated CIS or CISD code.

## Full CI

Let's start with a simple FullCI of water.

MultiPsi relies on Veloxchem to define the molecule and basis set. For more informations about how to run Veloxchem, please go to the Veloxchem webpage.

> <https://veloxchem.org>

Here we will start from a simple Hartree-Fock calculation in Veloxchem:

In [1]:
import veloxchem as vlx

water_str='''
O   0.0   0.0   0.0
H   0.0   1.4   1.1
H   0.0  -1.4   1.1
'''
water=vlx.Molecule.read_str(water_str)
basis = vlx.MolecularBasis.read(water,"6-31G")
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.compute(water, basis)

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 16.
                                                                                                                          
                                            Self Consistent Field Driver Setup                                            
                                                                                                                          
                   Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                   Initial Guess Model             : Superposition of Atomic Densities                                    
                   Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                   Max. Number of Iterations       : 50                                                                   
                   Max. Number of Erro

We then define an orbital space object to store the starting orbitals and define the type of CI expansion:

In [2]:
import multipsi as mtp

space=mtp.OrbSpace(water,scfdrv.mol_orbs)
print(space)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 5
Number of active orbitals:              0
Number of virtual orbitals:             8

    This is a restricted Hartree-Fock wavefunction


By default, OrbSpace defines a restricted Hartree-Fock wavefunction (or restricted open-shell if the molecule is not a singlet). To define the full CI, we simply use the command FCI().

In [3]:
space.FCI()
print(space)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 0
Number of active orbitals:              13
Number of virtual orbitals:             0

    This is a CASSCF wavefunction: CAS(10,13)


As we can see, internally the code assumes a complete active space, but since all orbitals are included in the active space, it is indeed a full CI.

We can also choose to freeze the core orbitals, as the correlation for core electrons tend to be constant and thus cancel out in relative energies.

In [4]:
space.FCI(nfrozen=1)
print(space)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 1
Number of active orbitals:              12
Number of virtual orbitals:             0

    This is a CASSCF wavefunction: CAS(8,12)


This reduces significantly the number of determinants and thus the calculation time.

We can now run the configuration interaction:

In [5]:
CIdrv=mtp.CIDriver(water,basis,space)
CIdrv.compute(1)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 1
Number of active orbitals:              12
Number of virtual orbitals:             0

    This is a CASSCF wavefunction: CAS(8,12)

          CI expansion:
          -------------
Number of determinants:      122760



        CI Iterations
        -------------

Iteration   Average energy      Time
     1        -75.81578151   0:00:00
     2        -75.89429264   0:00:00
     3        -75.90115019   0:00:00
     4        -75.90196355   0:00:00
     5        -75.90212958   0:00:00
     6        -75.90215048   0:00:00
     7        -75.90215439   0:00:00
     8        -75.90215510   0:00:00
     9        -75.90215521   0:00:00
    10        -75.90215524   0:00:00
    11        -75.90215524   0:00:00
Convergence reached in 11 iterations

        Final results
        -------------

* State 1
- Energy: -75.90215524152764
- S^2   : 0.00  (multiplicity = 1.0 )
- Natural orbitals


If we are interested in several states and not just the ground state, we simply provide the number of states to the compute function:

In [6]:
CIdrv.compute(3) #We want 3 states now


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 1
Number of active orbitals:              12
Number of virtual orbitals:             0

    This is a CASSCF wavefunction: CAS(8,12)

          CI expansion:
          -------------
Number of determinants:      122760



        CI Iterations
        -------------

Iteration   Average energy      Time
     1        -75.76916523   0:00:00
     2        -75.85009446   0:00:00
     3        -75.85786721   0:00:00
     4        -75.85873266   0:00:00
     5        -75.85890332   0:00:00
     6        -75.85892988   0:00:00
     7        -75.85893563   0:00:00
     8        -75.85893723   0:00:00
     9        -75.85893767   0:00:00
    10        -75.85893782   0:00:00
    11        -75.85893785   0:00:00
    12        -75.85893786   0:00:00
    13        -75.85893787   0:00:00
Convergence reached in 13 iterations

        Final results
        -------------

* State 1
- Energy: -7

At the end of the calculation, the CI Driver prints a few informations, but the key results can also be accessed directly in python.

In [7]:
print("Ground state energy:",CIdrv.getEnergy(0)) #Get the energy of a specific state, here state number 0
print("Array of all state energies:",CIdrv.getEnergies()) #Get a python list of energies
print()
print("Ground state natural orbitals occupation numbers:",CIdrv.getNON(0))

Ground state energy: -75.90215524245829
Array of all state energies: [-75.90215524245829, -75.8490306822384, -75.82562767830416]

Ground state natural orbitals occupation numbers: [1.98787 1.98052 1.72828 1.69443 0.30427 0.26590 0.01798 0.00977 0.00528
 0.00509 0.00035 0.00026]


## Truncated CI

To define a truncated CI, one simply needs to define the truncated expansion in the OrbSpace object.

CIS and CISD expansions have a specific command, and higher order truncations can be done by using CI(n) with n the excitation order.

In [8]:
space=mtp.OrbSpace(water,scfdrv.mol_orbs)
space.CISD(nfrozen=1) #Equivalent to space.CI(2,nfrozen=1)

CIdrv.compute(1)


          Active space definition:
          ------------------------
Number of inactive (occupied) orbitals: 1
Number of active orbitals:              12
Number of virtual orbitals:             0

    This is a CASSCF wavefunction: CAS(8,12)

          CI expansion:
          -------------
Number of determinants:      122760



        CI Iterations
        -------------

Iteration   Average energy      Time
     1        -75.81578151   0:00:00
     2        -75.89429264   0:00:00
     3        -75.90115019   0:00:00
     4        -75.90196355   0:00:00
     5        -75.90212958   0:00:00
     6        -75.90215048   0:00:00
     7        -75.90215439   0:00:00
     8        -75.90215510   0:00:00
     9        -75.90215521   0:00:00
    10        -75.90215524   0:00:00
    11        -75.90215524   0:00:00
Convergence reached in 11 iterations

        Final results
        -------------

* State 1
- Energy: -75.90215524152764
- S^2   : 0.00  (multiplicity = 1.0 )
- Natural orbitals
